# vLLM Speed UP Inference

In [1]:
import os

os.environ['VLLM_WORKER_MULTIPROC_METHOD'] = 'spawn'
from vllm import LLM, SamplingParams
max_model_len, tp_size = 4000, 1
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import numpy as np

def get_response_prompts(instructions):
    positive_prompts = []
    for instruction in instructions:
        messages = [
            {"role": "system", "content": "You are a helpful AI assistant."},
            {"role": "user", "content": instruction}
        ]
        positive_prompts.append(messages)
    return positive_prompts

def get_worse_response_question_prompts(instructions, positive_responses, worse_response_prompt_template):
    worse_prompts = []
    for instruction, response in zip(instructions, positive_responses):
        messages = [
            {"role": "system", "content": "You are a helpful AI assistant."},
            {"role": "user", "content": worse_response_prompt_template.format(input=instruction, generation=response)}
        ]
        worse_prompts.append(messages)
    return worse_prompts

def extract_negative_questions(instructions, positive_responses, inputs):
    all_negative_instructions = []
    temp_instructions = []
    temp_positive_responses = []
    for idx, item in enumerate(inputs):
        try:
            start = item.index("[The start of User Question]")+len("[The start of User Question] ")
            if("[The end of User Question]" in item):
                end = item.index("[The end of User Question]")
            else:
                end = len(item)
            question = item[start:end]
            all_negative_instructions.append(question)
            temp_instructions.append(instructions[idx])
            temp_positive_responses.append(positive_responses[idx])
        except:
            print("Generation String Not Found! ID:", idx)
            # print(negative)
    return temp_instructions, temp_positive_responses, all_negative_instructions

def extract_negative_responses(instructions, positive_responses, generated_text):
    all_responses = []
    for idx, (instruction, positive, negative) in enumerate(zip(instructions, positive_responses, generated_text)):
        try:
            start = negative.index("[The start of Modified Instruction Response]")+len("[The start of Modified Instruction Response] ")
            if("[The end of Modified Instruction Response]" in negative):
                end = negative.index("[The end of Modified Instruction Response]")
            else:
                end = len(negative)
            negative = negative[start:end]
            all_responses.append({"instruction": instruction, "positive": positive, "negative": negative})
        except:
            print("Generation String Not Found! ID:", idx)
            # print(negative)
    return all_responses

def create_judge_response_prompts(instructions, positives, negatives, judging_template):
    judge_prompts = []
    labels = []
    for instruction, positive, negative in zip(instructions, positives, negatives):
        rand_num = np.random.rand()
        if(rand_num < 0.5):
            messages = [
                {"role": "system", "content": "You are a helpful AI assistant."},
                {"role": "user", "content": judging_template.format(input=instruction, 
                                                                    generation=positive, 
                                                                    generation2=negative)},
            ]
            judge_prompts.append(messages)
            labels.append(1)
        else:
            messages = [
                {"role": "system", "content": "You are a helpful AI assistant."},
                {"role": "user", "content": judging_template.format(input=instruction, 
                                                                    generation=negative, 
                                                                    generation2=positive)},
            ]
            judge_prompts.append(messages)
            labels.append(0)
    return judge_prompts, labels

def load_textfile_as_string(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        return file.read()



In [2]:
model_name = "microsoft/Phi-3.5-mini-instruct"
# model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

In [3]:
messages = [
            {
                "role": "system",
                "content": "You are a helpful AI assistant."
            },
            {
                "role": "user",
                "content": "Please act as an impartial judge and evaluate the quality of the responses provided by two AI assistants to the user question displayed below. You should choose the assistant that follows the user's instructions and answers the user's question better. Begin your evaluation by first verifying whether each response contains any obvious or subtle errors. Then propose an appropriate evaluaiton rubric, e.g. 1-5 criteria that are important for evaluating responses to this specific user question. Continue your evaluation by checking each response carefully along those criteria. Based on the analysis in previous steps, choose which response is better overall. Avoid any position biases and ensure that the order in which the responses were presented does not influence your decision. Do not allow the length of the responses to influence your evaluation. Do not favor certain names of the assistants. Be as objective as possible. After providing your evaluation, output your final verdict by strictly following this format: \\\"[[A]]\\\" if assistant A is better, \\\"[[B]]\\\" if assistant B is better. \n\n[User Question]\nfast reading method when first few letters of words are bold\n\n[The Start of Assistant A's Answer]\nYou're referring to a technique called \"Rapid Serial Visual Presentation\" (RSVP) with bolded first few letters! This method can indeed help you read faster and more efficiently. Here's how it works:\n\n1. Identify the first few letters of each word that are bolded. This can be 2-3 letters, depending on your preference.\n2. Focus on the bolded letters to quickly identify the word. This helps your brain to recognize the word more quickly and accurately.\n3. Move your eyes to the next word, repeating the process of identifying the bolded letters and recognizing the word.\n4. Continue this process at a rapid pace, moving your eyes from one word to the next.\n\nThis technique can help you read faster in several ways:\n\n* Reduced eye movement: By focusing on the bolded letters, you reduce the amount of eye movement needed to read each word, allowing you to move more quickly through the text.\n* Increased recognition: The bolded letters help your brain to recognize words more quickly, as your brain is able to latch onto the familiar pattern of letters.\n* Improved processing: The bolded letters can also help your brain to process the words more efficiently, as the bolded letters provide a clear cue for your brain to recognize the word.\n\nTo get the most out of this technique, try the following:\n\n* Start with a moderate pace and gradually increase your speed as you become more comfortable with the technique.\n* Practice with different texts and materials to get a feel for how the technique works with different types of writing.\n* Experiment with different bolding patterns, such as bolding the first letter of each word or the first letter of every other word, to find what works best for you.\n\nRemember, the key to success with this technique is to practice regularly and develop a consistent reading rhythm. With time and practice, you can improve your reading speed and comprehension using this method!\n[The End of Assistant A's Answer]\n\n[The Start of Assistant B's Answer]\nYou're interested in learning a fast reading method that utilizes visual cues to improve recognition and processing! I'd be happy to help you with that.\n\nThe method I'd like to introduce you to is called the \"Meta-Guiding\" technique. It's a visual cue-based method that can help you read faster and more efficiently.\n\n**What is Meta-Guiding?**\n\nMeta-Guiding is a technique that uses visual cues to guide your eyes as you read. The goal is to train your eyes to move more efficiently across the page, recognizing words and phrases more quickly, and processing the information more effectively.\n\n**How to Implement Meta-Guiding:**\n\n1. **Start by choosing a text**: Select a passage that you want to read quickly and efficiently. It can be a short article, a chapter, or even a book.\n2. **Identify the visual cues**: As you read, pay attention to the visual cues that catch your attention. These can be:\n\t* Headings and subheadings\n\t* Bold or italicized text\n\t* Images or graphics\n\t* Quotes or dialogue\n\t* Numbered or bulleted lists\n3. **Use these cues as guides**: As you read, use these visual cues to guide your eyes across the page. For example, if you see a heading, use it as a reference point to scan the surrounding text.\n4. **Practice active reading**: As you read, ask yourself questions like:\n\t* What is the main idea of this section?\n\t* What is the author trying to say?\n\t* What are the key points being made?\n5. **Use your peripheral vision**: Instead of focusing on individual words, use your peripheral vision to take in larger chunks of text. This will help you recognize patterns and relationships between words.\n6. **Practice, practice, practice!**: The more you practice using visual cues to guide your reading, the more efficient and effective you'll become.\n\n**Benefits of Meta-Guiding:**\n\n1. **Faster reading speeds**: By using visual cues, you can train your eyes to move more quickly across the page, allowing you to read faster.\n2. **Improved comprehension**: By focusing on the main ideas and key points, you'll be able to comprehend the text more effectively.\n3. **Reduced eye fatigue**: By using your peripheral vision and focusing on visual cues, you'll reduce eye fatigue and strain.\n4. **Increased retention**: By actively engaging with the text and using visual cues, you'll be able to retain more information.\n\n**Tips and Variations:**\n\n1. **Start with short texts**: Begin with shorter texts and gradually move to longer ones as you become more comfortable with the technique.\n2. **Experiment with different types of text**: Try using Meta-Guiding with different types of text, such as articles, blog posts, or even social media updates.\n3. **Combine with other reading techniques**: You can combine Meta-Guiding with other reading techniques, such as skimming or scanning, to further improve your reading efficiency.\n\nI hope this helps you get started with the Meta-Guiding technique! Remember to practice regularly and have fun exploring the benefits of this visual cue-based method.\n[The End of Assistant B's Answer]"
            }
        ]
tokenizer = AutoTokenizer.from_pretrained(model_name)
prompt_token_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
print(prompt_token_ids)

<|system|>
You are a helpful AI assistant.<|end|>
<|user|>
Please act as an impartial judge and evaluate the quality of the responses provided by two AI assistants to the user question displayed below. You should choose the assistant that follows the user's instructions and answers the user's question better. Begin your evaluation by first verifying whether each response contains any obvious or subtle errors. Then propose an appropriate evaluaiton rubric, e.g. 1-5 criteria that are important for evaluating responses to this specific user question. Continue your evaluation by checking each response carefully along those criteria. Based on the analysis in previous steps, choose which response is better overall. Avoid any position biases and ensure that the order in which the responses were presented does not influence your decision. Do not allow the length of the responses to influence your evaluation. Do not favor certain names of the assistants. Be as objective as possible. After provi

In [4]:


torch.random.manual_seed(0)


sampling_params = SamplingParams(temperature=0.8, top_p=0.95, max_tokens=2000)
llm = LLM(model=model_name, tensor_parallel_size=tp_size, max_model_len=max_model_len, 
          trust_remote_code=True, enforce_eager=True, gpu_memory_utilization=0.90)


WARNING 12-30 18:33:03 config.py:383] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 12-30 18:33:03 llm_engine.py:232] Initializing an LLM engine (v0.6.1) with config: model='microsoft/Phi-3.5-mini-instruct', speculative_config=None, tokenizer='microsoft/Phi-3.5-mini-instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=4000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, c

/home/pradyumnachari/anaconda3/envs/llm/lib/python3.11/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/pradyumnachari/anaconda3/envs/llm/lib/python3.11/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")


INFO 12-30 18:33:05 model_runner.py:997] Starting to load model microsoft/Phi-3.5-mini-instruct...
INFO 12-30 18:33:06 selector.py:240] Cannot use FlashAttention-2 backend due to sliding window.
INFO 12-30 18:33:06 selector.py:116] Using XFormers backend.
INFO 12-30 18:33:06 weight_utils.py:242] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 12-30 18:33:08 model_runner.py:1008] Loading model weights took 7.1659 GB
INFO 12-30 18:33:09 gpu_executor.py:122] # GPU blocks: 2325, # CPU blocks: 682


In [103]:
question = "Give me a brief history of the Roman Empire."
question = "What are the benefits of stretching?"
question = "Write me a python function that outputs the number of parameters in a pytorch model."
o_question = "A train leaves Station A at 8:00 AM, traveling at 60 km/h. Another train departs Station B, 120 km away from Station A, at 9:00 AM, traveling toward Station A at 80 km/h. At what time do the two trains meet?"
# o_question = """The island of Solara is home to a unique species of bird called the Solara Finch. These finches primarily feed on a specific type of berry that grows on tall bushes. During the dry season, the berry supply diminishes, forcing the finches to travel farther to find food. Researchers have observed that during the dry season, the finches’ wing muscles become stronger, allowing them to fly greater distances. This adaptation enables them to survive harsh conditions. However, researchers are concerned that prolonged dry seasons due to climate change may outpace the finches' ability to adapt.

# Question: Based on the passage, which of the following is the most likely outcome if the dry season becomes significantly longer over time?

# A) The Solara Finch population will increase due to improved flight capabilities.
# B) The Solara Finch population may decline because they might not adapt quickly enough.
# C) The finches will switch to a diet of insects to compensate for the loss of berries.
# D) The finches will migrate permanently to areas where berries are abundant year-round."""
# o_question = """In recent years, the rapid advancement of artificial intelligence (AI) has sparked debates about its ethical implications and societal impact. Proponents argue that AI can solve complex problems, from predicting climate patterns to diagnosing diseases, potentially improving countless lives. Critics, however, warn of potential job displacement, biased algorithms, and privacy concerns. Some researchers advocate for stronger regulatory frameworks to ensure AI development aligns with human values, while others fear that excessive regulation may stifle innovation and delay technological progress.

# Question:
# Discuss the potential trade-offs involved in implementing stricter regulations on artificial intelligence development, considering both ethical and societal perspectives."""

In [104]:
generated_responses_pos_prompts = [[
            {
                "role": "system",
                "content": "You are a helpful AI assistant."
            },
            {
                "role": "user",
                "content": o_question
            }
        ]]
prompt_token_ids = [tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=True) for messages in generated_responses_pos_prompts]
outputs = llm.generate(prompt_token_ids=prompt_token_ids, sampling_params=sampling_params)
generated_responses_pos = [output.outputs[0].text for output in outputs]
print(generated_responses_pos[0])

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.56s/it, est. speed input: 14.93 toks/s, output: 53.42 toks/s]

 Let's break down this problem step-by-step:

1. In one hour (from 8:00 AM to 9:00 AM), the first train travels 60 km.
2. The distance between the two trains after the first train has traveled for one hour is now 120 km - 60 km = 60 km.
3. Now, both trains are moving towards each other. Their combined speed is 60 km/h + 80 km/h = 140 km/h.
4. To cover the remaining 60 km between them, they need to travel at a combined speed of 140 km/h.
5. To find out how long it takes them to cover the remaining distance, we can use the formula: time = distance / speed.
6. In this case, time = 60 km / 140 km/h = 0.4286 hours (approximately).
7. Since the second train left at 9:00 AM, they will meet around 9:00 AM + 0.4286 hours = 9:26 AM.

So, the two trains meet at approximately 9:26 AM.

Final answer: The two trains meet at around 9:26 AM.


In [102]:
# Judgement
judgement_prompt = load_textfile_as_string("../prompts/judgement.prompt")
formatted_prompt = judgement_prompt.format(question=o_question, response=generated_responses_pos[0])

generated_responses_pos_prompts = [[
            {
                "role": "system",
                "content": "You are a helpful AI assistant."
            },
            {
                "role": "user",
                "content": formatted_prompt
            }
        ]]
prompt_token_ids = [tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=True) for messages in generated_responses_pos_prompts]
outputs = llm.generate(prompt_token_ids=prompt_token_ids, sampling_params=sampling_params)
outs = [output.outputs[0].text for output in outputs]
judgement = outs[0]
print(judgement)

Processed prompts: 100%|██████████| 1/1 [00:15<00:00, 15.16s/it, est. speed input: 45.46 toks/s, output: 50.67 toks/s]

 The response provided by the AI assistant contains a calculation error in determining the meeting time of the two trains. Let's correct this and evaluate the response based on the following criteria:

1. Accuracy of calculations and logic applied
2. Clarity and coherence in the explanation
3. Relevance and precision of the information provided
4. Correctness of rounding (if applicable)

Revised Calculation:

1. From 8:00 AM to 9:00 AM, the first train covers 60 km. The remaining distance between the two trains is now 120 km - 60 km = 60 km.
2. The relative speed at which the two trains are approaching each other is 60 km/h + 80 km/h = 140 km/h.
3. To find out how long it takes for the two trains to cover the remaining 60 km at a combined speed of 140 km/h, we use the formula: Time = Distance / Speed.
4. Time = 60 km / 140 km/h ≈ 0.43 hours (approximately 26 minutes).
5. Since the second train started at 9:00 AM, adding approximately 26 minutes to this time gives us 9:26 AM.
6. During 

In [101]:
suggestion_prompt = load_textfile_as_string("../prompts/suggest.prompt")
generated_responses_pos_prompts = [[
            {
                "role": "system",
                "content": "You are a helpful AI assistant."
            },
            {
                "role": "user",
                "content": suggestion_prompt.format(question=o_question, response=generated_responses_pos[0])
            }
        ]]
prompt_token_ids = [tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=True) for messages in generated_responses_pos_prompts]
outputs = llm.generate(prompt_token_ids=prompt_token_ids, sampling_params=sampling_params)
generated_responses = [output.outputs[0].text for output in outputs]
print(generated_responses[0])

Processed prompts: 100%|██████████| 1/1 [00:14<00:00, 14.85s/it, est. speed input: 48.42 toks/s, output: 51.31 toks/s]

 The response provided contains a logical flow and attempts to address the problem with relevant calculations. However, there are mistakes in the calculation and final time estimation. Here's an evaluation of the response:

1. Clarity: The explanation is generally clear, but there are some calculation errors.
2. Conciseness: The response is concise but includes unnecessary approximations.
3. Precision and Accuracy: The calculation lacks precision. The error occurs when the trains' combined distance traveled is incorrectly calculated.
4. Relevance: The response stays relevant to the question throughout.
5. Engagement: The response does not engage the reader with any interesting facts or context.
6. Depth and Breadth: The response covers the basic steps but lacks depth in explanation and does not explore alternative methods or potential pitfalls.
7. Logical Flow: The logical flow is disrupted by the calculation errors.
8. Tone and Objectivity: The tone is informative, but the response la

In [92]:
question = "Give me a detailed analysis of the political structure and key events during the Roman Republic (509-27 BCE)."
question = "A train leaves Station A at 8:00 AM, traveling at 60 km/h. Another train departs Station B, 120 km away from Station A, at 9:00 AM, traveling towards Station A at 80 km/h. At what time will the two trains meet, rounded to the nearest minute?"
question = "Discuss the ethical and societal trade-offs specifically related to job displacement and economic impact when implementing stricter regulations on artificial intelligence development. How might these regulations affect innovation and global competitiveness in the tech industry?"
generated_responses_pos_prompts = [[
            {
                "role": "system",
                "content": "You are a helpful AI assistant."
            },
            {
                "role": "user",
                "content": question
            }
        ]]
prompt_token_ids = [tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=True) for messages in generated_responses_pos_prompts]
outputs = llm.generate(prompt_token_ids=prompt_token_ids, sampling_params=sampling_params)
generated_responses_pos = [output.outputs[0].text for output in outputs]
print(generated_responses_pos[0])

Processed prompts: 100%|██████████| 1/1 [00:17<00:00, 17.05s/it, est. speed input: 3.75 toks/s, output: 52.50 toks/s]

 The implementation of stricter regulations on artificial intelligence (AI) development involves complex ethical and societal trade-offs, particularly concerning job displacement and economic impact, as well as the potential effects on innovation and global competitiveness in the tech industry. Here are some key considerations:

### Ethical and Societal Trade-offs

**1. Job Displacement:**
- **Pros:**
    - Strict regulations may lead to the development of more ethically aligned AI systems that prioritize human-centric values and thus, potentially create jobs in ethically compliant sectors.
    - Regulations might encourage a shift towards higher-skilled positions as routine tasks are automated.
- **Cons:**
    - AI can perform a wide range of tasks with efficiency and precision, leading to significant job displacement in various industries.
    - The immediate loss of jobs may exacerbate economic inequality and cause hardship for individuals and communities heavily reliant on routine,

In [67]:
# Judgement
judgement_prompt = load_textfile_as_string("../prompts/judgement.prompt")
formatted_prompt = judgement_prompt.format(question=o_question, response=generated_responses_pos[0])

generated_responses_pos_prompts = [[
            {
                "role": "system",
                "content": "You are a helpful AI assistant."
            },
            {
                "role": "user",
                "content": formatted_prompt
            }
        ]]
prompt_token_ids = [tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=True) for messages in generated_responses_pos_prompts]
outputs = llm.generate(prompt_token_ids=prompt_token_ids, sampling_params=sampling_params)
outs = [output.outputs[0].text for output in outputs]
judgement = outs[0]
print(judgement)

Processed prompts: 100%|██████████| 1/1 [00:13<00:00, 13.30s/it, est. speed input: 94.09 toks/s, output: 50.31 toks/s]

 The provided response effectively addresses the question by outlining several key trade-offs involved in implementing stricter regulations on artificial intelligence (AI) development. The response is structured and touches upon the ethical and societal perspectives requested. Here is an evaluation using a proposed rubric:

**Rubric for Evaluation:**

1. **Clarity of Expression:**
   - The response is well-articulated and clear.
   - Criteria: 5/5

2. **Depth of Analysis:**
   - The response explores various dimensions of the trade-offs involved, including job displacement, economic impact, innovation, and global competitiveness.
   - Criteria: 5/5

3. **Relevance to Question:**
   - The response directly addresses the potential trade-offs in stricter AI regulations from ethical and societal perspectives.
   - Criteria: 5/5

4. **Consideration of Multiple Perspectives:**
   - The response acknowledges both the advantages and disadvantages, offering a balanced view.
   - Criteria: 5/5



In [68]:
generated_responses_pos_prompts = [[
            {
                "role": "system",
                "content": "You are a helpful AI assistant."
            },
            {
                "role": "user",
                "content": suggestion_prompt.format(question=o_question, response=generated_responses_pos[0])
            }
        ]]
prompt_token_ids = [tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=True) for messages in generated_responses_pos_prompts]
outputs = llm.generate(prompt_token_ids=prompt_token_ids, sampling_params=sampling_params)
generated_responses = [output.outputs[0].text for output in outputs]
print(generated_responses[0])

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.77s/it, est. speed input: 324.45 toks/s, output: 49.65 toks/s]

 The response is well-structured and directly addresses the potential trade-offs involved in implementing stricter regulations on AI from both ethical and societal perspectives. The response clearly outlines the pros and cons in each area, providing a comprehensive overview of the complexities at play. To ensure that the response aligns precisely with the user question, the structure and scope are well-maintained, with a balanced view that considers both the benefits and drawbacks of regulation.

No change to the user question is necessary as it effectively prompts a discussion of the trade-offs associated with AI regulation, which the response thoroughly explores. The question is clear and well-suited to elicit the provided detailed analysis of ethical and societal considerations. The response fully satisfies the requirements of the user's query without needing further modification.


In [ ]:
from datasets import load_dataset
from tqdm import tqdm
import pickle

#dataset is a subset of WildChat
dataset = load_dataset("facebook/Self-taught-evaluator-DPO-data")
WildChat = load_dataset("allenai/WildChat-1M")

# get hashes
#load the hash_id2content dictionary
# with open("hash_id2content.pkl", "rb") as f:
#   hash_id2content = pickle.load(f)
hash_id2content = dict()
for ex in tqdm(WildChat["train"], total=len(WildChat["train"])):
  turn = ex["turn"]
  hash_id2content[ex["conversation_hash"]] = ex["conversation"][2 * (turn - 1)]["content"]


print("Starting 2.")
train_data = []
for ex in tqdm(dataset["train"], total=len(dataset["train"])):
  if ex["instruction"] not in hash_id2content:
    continue
  else:
    ex["src"] = ex["src"].replace(ex["instruction"], hash_id2content[ex["instruction"]])
    train_data.append(ex)

print("Starting 3.")
# Extract Instructions
skip = 6 # I found that the dataset instructions has the same instruction repeated 6 times? Not sure why. 
all_instructions = []
num_responses = len(dataset['train'])//skip
for i in tqdm(range(0, num_responses*skip, skip), total=num_responses):
    try:
        instruction_example = hash_id2content[dataset['train'][i]['instruction']]
        all_instructions.append(instruction_example)
    except:
        continue
print(all_instructions[0])

In [ ]:
worse_response_prompt_template = load_textfile_as_string('./prompts/worse_response_v2.prompt')
print(worse_response_prompt_template)
judging_prompt_template = load_textfile_as_string('./prompts/eval_plan.prompt')
print(judging_prompt_template)

In [ ]:
generated_responses_pos_prompts = get_response_prompts(all_instructions)
prompt_token_ids = [tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=True) for messages in generated_responses_pos_prompts]
outputs = llm.generate(prompt_token_ids=prompt_token_ids, sampling_params=sampling_params)

generated_responses_pos = [output.outputs[0].text for output in outputs]
print(generated_responses_pos[0])

In [ ]:
all_instructions_pos_neg = get_worse_response_question_prompts(all_instructions, generated_responses_pos, worse_response_prompt_template)
prompt_token_ids = [tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=True) for messages in all_instructions_pos_neg]
outputs = llm.generate(prompt_token_ids=prompt_token_ids, sampling_params=sampling_params)

all_instructions_neg = [output.outputs[0].text for output in outputs]
print(all_instructions_neg[0])

In [ ]:
all_instructions[0]

In [ ]:
generated_responses_pos[0]

In [ ]:
all_instructions_neg[0]

In [ ]:
f_all_instructions, f_generated_responses_pos, f_all_instructions_neg = extract_negative_questions(all_instructions, generated_responses_pos, all_instructions_neg)

In [ ]:
len(all_instructions), len(f_all_instructions), len(f_generated_responses_pos), len(f_all_instructions_neg)

In [ ]:
# items_to_judge = extract_negative_responses(all_instructions, generated_responses_pos, generated_responses_neg)
# print(items_to_judge[0])
# print(len(items_to_judge))

generated_responses_neg_prompts = get_response_prompts(f_all_instructions_neg)
prompt_token_ids = [tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=True) for messages in generated_responses_neg_prompts]
outputs = llm.generate(prompt_token_ids=prompt_token_ids, sampling_params=sampling_params)

f_generated_responses_neg = [output.outputs[0].text for output in outputs]
print(f_generated_responses_neg[0])

In [ ]:
all_judgements, labels = create_judge_response_prompts(f_all_instructions, f_generated_responses_pos, f_generated_responses_neg, judging_prompt_template)
prompt_token_ids = [tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=True) for messages in all_judgements]
outputs = llm.generate(prompt_token_ids=prompt_token_ids, sampling_params=sampling_params)

generated_judgements = [output.outputs[0].text for output in outputs]
print(generated_judgements[0])


In [ ]:
idx = 0
print(f_all_instructions[idx])
print("_____"*10)
print(f_generated_responses_pos[idx])
print("_____"*10)
print(f_all_instructions_neg[idx])
print("_____"*10)
print(f_generated_responses_neg[idx])


In [ ]:
count_correct = 0
A_correct = []
B_correct = []
count_correct_format_incorrect_answer = 0
for judge_prompt, judgement, label in zip(all_judgements, generated_judgements, labels):
    if(("[[A]]" in judgement and label == 1) or ("[[B]]" in judgement and label == 0)):
        count_correct += 1
        if("[[A]]" in judgement):
            A_correct.append([{"role": "system", "content": "You are a helpful AI assistant."}, 
                                {"role": "user", "content": judge_prompt[1]["content"]},
                                {"role": "assistant", "content": judgement}])
        else:
            B_correct.append([{"role": "system", "content": "You are a helpful AI assistant."}, 
                                {"role": "user", "content": judge_prompt[1]["content"]},
                                {"role": "assistant", "content": judgement}])
    else:
        # print(judgement)
        # print("*************")
        if("[[A]]" in judgement or "[[B]]" in judgement):
            count_correct_format_incorrect_answer += 1
print("Correct", count_correct, "/", len(all_judgements))
print("A correct: ", len(A_correct))
print("B correct: ", len(B_correct))
print("Formatted Incorrect: ", count_correct_format_incorrect_answer, "/", len(all_judgements))

In [ ]:
# Balance the dataset
import random
if(len(A_correct) > len(B_correct)):
    A_correct_sampled = random.sample(A_correct, len(B_correct))
    B_correct_sampled = B_correct
else:
    B_correct_sampled = random.sample(B_correct, len(A_correct))
    A_correct_sampled = A_correct

In [ ]:
len(A_correct_sampled), len(B_correct_sampled)

In [ ]:
#calculate token statistics for the generated responses
tokenizer = AutoTokenizer.from_pretrained(model_name)

def calculate_token_statistics(messages):
    token_counts = []
    for message in messages:
        tokens = tokenizer.apply_chat_template(message, add_generation_prompt=False, tokenize=True)
        token_counts.append(len(tokens))
    print("Mean Tokens: ", np.mean(token_counts))
    print("Median Tokens: ", np.median(token_counts))
    print("Max Tokens: ", np.max(token_counts))
    print("Min Tokens: ", np.min(token_counts))
    print("STD Tokens: ", np.std(token_counts))

calculate_token_statistics(A_correct_sampled+ B_correct_sampled)

In [ ]:
# Save the dataset
import json

all_sft_samples = A_correct_sampled + B_correct_sampled
json_array = []
for i, judgement in enumerate(all_sft_samples):
    json_array.append({"messages": judgement})

# Define the output file path
output_file_path = 'all_judgements_llama.json'

# Write the list of dictionaries to the JSON file
with open(output_file_path, 'w', encoding='utf-8') as f:
    json.dump(json_array, f, ensure_ascii=False, indent=4)